In [10]:
import logging
import os
import sys
import shutil
import tempfile

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pandas as pd

from torch.utils.tensorboard import SummaryWriter
import numpy as np
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from models.resnet import resnet18 

import monai
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import DataLoader, ImageDataset
from monai.transforms import (
    EnsureChannelFirst,
    Compose,
    RandRotate90,
    Resize,
    ScaleIntensity,
)


pin_memory = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# print_config()



#### **1. ResNet18: El Caballo de Batalla Generalista**

**ResNet18** es una arquitectura de red neuronal convolucional muy popular, parte de la familia Residual Networks (ResNet). Su innovación principal radica en el uso de **"bloques residuales"** o "saltos" (skip connections). Estos saltos permiten que la información y los gradientes fluyan más fácilmente a través de muchas capas, resolviendo el problema del "gradiente desvanecido" que dificultaba el entrenamiento de redes muy profundas.

* **Pre-entrenamiento:** Comúnmente, ResNet18 se pre-entrena en el dataset **ImageNet**. Este es un gigantesco conjunto de datos con millones de imágenes de objetos cotidianos (perros, coches, sillas, etc.) de 1000 categorías diferentes.
* **Ventajas en imágenes médicas:** A pesar de haber sido entrenado con imágenes naturales, las características de bajo nivel que ResNet18 aprende de ImageNet (detección de bordes, patrones de textura) son sorprendentemente útiles como punto de partida para el análisis de imágenes médicas. Es un excelente punto de inicio general.
* **Consideración:** Puede haber una "brecha de dominio" entre las imágenes de ImageNet y las imágenes médicas, lo que significa que el fine-tuning es crucial para adaptar el modelo a las particularidades de los datos médicos (contrastes, resoluciones, tipos de ruido).

---

#### **2. MedicalNet: Especialización para el Dominio Médico**

**MedicalNet** es una iniciativa que proporciona modelos (incluyendo variantes de ResNet como ResNet18) que han sido **pre-entrenados específicamente en un vasto y diverso conjunto de datos de imágenes médicas**.

* **Pre-entrenamiento:** A diferencia de ResNet18 estándar, MedicalNet ha sido entrenado con millones de imágenes provenientes de diversas modalidades médicas (MRI, CT, rayos X, ultrasonido) y cubriendo diferentes órganos y patologías.
* **Ventajas en imágenes médicas:**
    * **Mayor relevancia de las características:** Al estar pre-entrenado en datos médicos, MedicalNet ya ha aprendido patrones y características que son intrínsecamente más relevantes para el diagnóstico y análisis clínico.
    * **Menor brecha de dominio:** Esto puede traducirse en un mejor rendimiento inicial, una convergencia más rápida durante el fine-tuning y, potencialmente, un mejor rendimiento final con menos datos de entrenamiento específicos para tu tarea.
    * **Adaptado a diferentes modalidades:** Su entrenamiento diverso lo hace robusto para trabajar con distintos tipos de imágenes médicas.



In [2]:
# Dimensiones de tus volúmenes de MRI usamos 256x256x256 ya que es un tamaño común para imágenes de resonancia magnética cerebral en freesurfer
input_D = 256
input_H = 256
input_W = 256

# Número de canales (1 para norm.mgz)
input_C = 1 
model = resnet18(sample_input_D=input_D,
                 sample_input_H=input_H,
                 sample_input_W=input_W,
                 num_seg_classes=1)  # ATENTO A ESTO PORQUE ES IMPORTANTE PARA LA CLASIFICACION

/home/nicolas/Documentos/gliara/models/resnet.py:173: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


Pesos de MedicalNet y ResNet18:
https://share.weiyun.com/55sZyIx 

In [3]:
pretrained_weights_path = "pretrain/resnet_18_23dataset.pth"

try:
    state_dict = torch.load(pretrained_weights_path)
    model.load_state_dict(state_dict, strict=False)
    print(f"Pesos de ResNet-18 cargados exitosamente desde {pretrained_weights_path}")

except FileNotFoundError:
    print(f"Error: El archivo de pesos no se encontró en {pretrained_weights_path}")
    print("Asegúrate de que la ruta sea correcta y el archivo exista.")
except Exception as e:
    print(f"Ocurrió un error al cargar los pesos: {e}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Modelo movido a: {device}")

/tmp/ipykernel_61891/2331833451.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(pretrained_weights_path)


Error: El archivo de pesos no se encontró en pretrain/resnet_18_23dataset.pth
Asegúrate de que la ruta sea correcta y el archivo exista.
Modelo movido a: cuda


In [4]:
# Define el número de clases para tu tarea de clasificación binaria (Alzheimer sí/no)
your_num_classes = 2
final_conv_layer = model.conv_seg[0]
num_in_features = final_conv_layer.in_channels
model.conv_seg = nn.Sequential(
    nn.Conv3d(num_in_features, your_num_classes,
              kernel_size=final_conv_layer.kernel_size,
              stride=final_conv_layer.stride,
              padding=final_conv_layer.padding)
)

print(f"Capa de salida (conv_seg) adaptada a {your_num_classes} clases para el proyecto Gliara.")

# Mueve el modelo completo (incluida la nueva capa) a la GPU
model.to(device)
print(f"Modelo movido a: {device}")


Capa de salida (conv_seg) adaptada a 2 clases para el proyecto Gliara.
Modelo movido a: cuda


# Preparar la data

In [20]:
Alz = pd.read_csv("Alz.csv")

,ID,M/F,Hand,Age,Educ,SES,MMSE,CDR,eTIV,nWBV,ASF,Delay,Alzheimer
0,OAS1_0001_MR1,F,R,74,2.0,3.0,29.0,0.0,1344,0.743,1.306,NaN,0
1,OAS1_0002_MR1,F,R,55,4.0,1.0,29.0,0.0,1147,0.810,1.531,NaN,0
2,OAS1_0010_MR1,M,R,74,5.0,2.0,30.0,0.0,1636,0.689,1.073,NaN,0
3,OAS1_0011_MR1,F,R,52,3.0,2.0,30.0,0.0,1321,0.827,1.329,NaN,0
4,OAS1_0013_MR1,F,R,81,5.0,2.0,30.0,0.0,1664,0.679,1.055,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,OAS1_0425_MR1,F,R,78,1.0,4.0,23.0,1.0,1461,0.715,1.201,NaN,1
231,OAS1_0430_MR1,M,R,71,4.0,1.0,17.0,1.0,1562,0.687,1.123,NaN,1
232,OAS1_0452_MR1,M,R,75,1.0,4.0,22.0,1.0,1656,0.762,1.060,NaN,1
233,OAS1_0308_MR1,F,R,78,3.0,3.0,15.0,2.0,1401,0.703,1.253,NaN,1


# Entrenamiento

In [9]:
criterion = nn.CrossEntropyLoss()
LEARNING_RATE = 1e-4
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

NUM_EPOCHS = 30

In [ ]:
best_val_loss = float('inf') # Para guardar el mejor modelo
best_epoch_accuracy = 0.0 # Para guardar la mejor precisión de validación
model_save_path = 'best_medicalnet_resnet18_gliara_model.pth' # Ruta para guardar el modelo

print("\n--- Comenzando el Fine-Tuning para Gliara ---")

for epoch in range(NUM_EPOCHS):
    # --- Fase de Entrenamiento ---
    model.train() # Pone el modelo en modo entrenamiento
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    # Iterar sobre los lotes de datos de entrenamiento
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device) # Mueve inputs a la GPU
        labels = labels.to(device) # Mueve labels a la GPU

        # Poner a cero los gradientes del optimizador para evitar acumulación
        optimizer.zero_grad()

        # Forward pass: El modelo hace predicciones
        outputs = model(inputs)

        # Calcular la pérdida
        loss = criterion(outputs, labels)

        # Backward pass: Calcular gradientes
        loss.backward()

        # Optimización: Actualizar los pesos del modelo
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

        # Calcular precisión (para clasificación)
        _, predicted = torch.max(outputs.data, 1) # Obtiene la clase con mayor probabilidad
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    # Calcular pérdida y precisión promedio por época
    epoch_train_loss = running_loss / len(train_dataset)
    epoch_train_accuracy = correct_predictions / total_samples if total_samples > 0 else 0

    print(f"Época {epoch+1}/{NUM_EPOCHS} - Entrenamiento | Pérdida: {epoch_train_loss:.4f} | Precisión: {epoch_train_accuracy:.4f}")

    # --- Fase de Validación ---
    model.eval() # Pone el modelo en modo evaluación (desactiva dropout, etc.)
    val_running_loss = 0.0
    val_correct_predictions = 0
    val_total_samples = 0

    with torch.no_grad(): # Desactiva el cálculo de gradientes para ahorrar memoria y tiempo
        for batch_idx, (inputs, labels) in enumerate(val_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs.data, 1)
            val_total_samples += labels.size(0)
            val_correct_predictions += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(val_dataset)
    val_epoch_accuracy = val_correct_predictions / val_total_samples if val_total_samples > 0 else 0

    print(f"Época {epoch+1}/{NUM_EPOCHS} - Validación  | Pérdida: {val_epoch_loss:.4f} | Precisión: {val_epoch_accuracy:.4f}")

    # --- Paso del Learning Rate Scheduler ---
    scheduler.step(val_epoch_loss) # Se ajustará la LR si la pérdida de validación no mejora

    # --- Guardar el mejor modelo ---
    # Guardamos el modelo si la pérdida de validación mejora
    if val_epoch_loss < best_val_loss:
        best_val_loss = val_epoch_loss
        best_epoch_accuracy = val_epoch_accuracy
        torch.save(model.state_dict(), model_save_path)
        print(f"¡Modelo guardado! Pérdida de validación mejorada a {best_val_loss:.4f}")

print("\n--- Fine-tuning completado para Gliara ---")
print(f"Mejor pérdida de validación: {best_val_loss:.4f}")
print(f"Mejor precisión de validación: {best_epoch_accuracy:.4f}")
print(f"Modelo final guardado en: {model_save_path}")